In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# -----------------------
# 1. Load Dataset
# -----------------------
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx"
data = pd.read_excel(url)

# Rename columns
data.columns = [
    'Relative_Compactness', 'Surface_Area', 'Wall_Area', 'Roof_Area',
    'Overall_Height', 'Orientation', 'Glazing_Area', 'Glazing_Area_Distribution',
    'Heating_Load', 'Cooling_Load'
]

# Features & Target
X = data.iloc[:, :-2]
y = data.iloc[:, -2:]

# -----------------------
# 2. Train Test Split + Scaling
# -----------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# -----------------------
# 3. Train Models
# -----------------------
results = {}

# Linear Regression
lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled, y_train)
y_pred_lr = lin_reg.predict(X_test_scaled)

results['Linear Regression'] = {
    "RMSE_Heating": np.sqrt(mean_squared_error(y_test.iloc[:,0], y_pred_lr[:,0])),
    "R2_Heating": r2_score(y_test.iloc[:,0], y_pred_lr[:,0]),
    "RMSE_Cooling": np.sqrt(mean_squared_error(y_test.iloc[:,1], y_pred_lr[:,1])),
    "R2_Cooling": r2_score(y_test.iloc[:,1], y_pred_lr[:,1]),
}

# Support Vector Regressor
svr = MultiOutputRegressor(SVR(kernel='rbf'))
svr.fit(X_train_scaled, y_train)
y_pred_svr = svr.predict(X_test_scaled)

results['SVR'] = {
    "RMSE_Heating": np.sqrt(mean_squared_error(y_test.iloc[:,0], y_pred_svr[:,0])),
    "R2_Heating": r2_score(y_test.iloc[:,0], y_pred_svr[:,0]),
    "RMSE_Cooling": np.sqrt(mean_squared_error(y_test.iloc[:,1], y_pred_svr[:,1])),
    "R2_Cooling": r2_score(y_test.iloc[:,1], y_pred_svr[:,1]),
}

# Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)

results['Random Forest'] = {
    "RMSE_Heating": np.sqrt(mean_squared_error(y_test.iloc[:,0], y_pred_rf[:,0])),
    "R2_Heating": r2_score(y_test.iloc[:,0], y_pred_rf[:,0]),
    "RMSE_Cooling": np.sqrt(mean_squared_error(y_test.iloc[:,1], y_pred_rf[:,1])),
    "R2_Cooling": r2_score(y_test.iloc[:,1], y_pred_rf[:,1]),
}

# -----------------------
# 4. Results Table
# -----------------------
results_df = pd.DataFrame(results).T
print(results_df)


                   RMSE_Heating  R2_Heating  RMSE_Cooling  R2_Cooling
Linear Regression      3.025427    0.912185      3.145382    0.893226
SVR                    2.823091    0.923538      3.259460    0.885340
Random Forest          0.494133    0.997657      1.925468    0.959988
